In [ ]:
import pandas as pd
from unidecode import unidecode
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import glob

In [ ]:
# path que contiene los csv
folder_path = './sentimientos/arkadia/join/'  # Change this to your folder path

# listar todos los csv
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# append de los csv
df_list = []
for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

# concatenar csv
merged_df = pd.concat(df_list, ignore_index=True)

# guardar
output_file = './sentimientos/arkadia/join/sentiments_competencia_arkadia.csv'  # Name your output file
merged_df.to_csv(output_file, index=False)


In [ ]:
#Unidecode texto de reviews y nombres

df = pd.read_csv('./sentimientos/arkadia/santafe_arkadia.csv')
reviews_gmaps = df[['review_text','rating','review_timestamp', 'google_id','name','review_likes']]
for column in df.columns:
    reviews_gmaps['review_text'] = reviews_gmaps['review_text'].apply(lambda x: unidecode(str(x)) if isinstance(x, str) else x).astype(str)
    reviews_gmaps['name'] = reviews_gmaps['name'].apply(lambda x: unidecode(str(x)) if isinstance(x, str) else x).astype(str)

In [ ]:
import pandas as pd
from transformers import pipeline

# Cargar un modelo pre-entrenado y la pipeline de análisis de sentimiento
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Definir una función que aplica el análisis de sentimiento
def analyze_sentiment(text):
    result = sentiment_pipeline(text)[0]
    return result['label']

# Aplicar la función al DataFrame
reviews_gmaps['sentiment'] = reviews_gmaps['review_text'].apply(analyze_sentiment)

reviews_gmaps

In [ ]:
reviews_gmaps.to_csv('./sentimientos/arkadia/sentiments_santafe_arkadia.csv', index = False)

## Reviews per hour

In [ ]:
#Timestamp reviews

reviews_gmaps = pd.read_csv('./sentimientos/arkadia/join/sentiments_competencia_arkadia.csv')
reviews_gmaps['date'] = pd.to_datetime(reviews_gmaps['review_timestamp'], unit='s')
reviews_gmaps['timestamp'] = reviews_gmaps['date'].dt.strftime('%Y-%m-%d %H:%M:%S').astype(str)
reviews_gmaps['hour'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[1].split(':')[0].strip(), axis=1)
reviews_gmaps['year'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[0].split('-')[0].strip(), axis=1)
reviews_gmaps['month'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[0].split('-')[1].strip(), axis=1)

In [ ]:
conteos = reviews_gmaps['name'].value_counts()

# Convertimos el conteo en un DataFrame.
df_conteos = conteos.reset_index()
df_conteos.columns = ['name', 'total']

# Ahora df_conteos es el DataFrame que contiene los nombres y el total de veces que se repiten.
print(df_conteos)
df_conteos.to_csv(f'./sentimientos/valuecount/fontanar/valuecount_santafe_fontanar.csv', index = False)

In [ ]:
ds = ["%.2d" % i for i in range(7,12)][1:]

In [ ]:
# Contamos las ocurrencias de cada nombre en la columna 'name'.
for d_ in ds:
    
    reviews_month = reviews_gmaps.loc[reviews_gmaps['month'] == d_]
    conteos = reviews_month['name'].value_counts()

    # Convertimos el conteo en un DataFrame.
    df_conteos = conteos.reset_index()
    df_conteos.columns = ['name', 'total']

    # Ahora df_conteos es el DataFrame que contiene los nombres y el total de veces que se repiten.
    print(df_conteos)
    df_conteos.to_csv(f'./sentimientos/valuecount/arkadia/valuecount_arkadia_{d_}.csv', index = False)

In [ ]:
# extracting hour of the review
reviews_gmaps = pd.read_csv('./sentimientos/sentiments_arkadia.csv')
reviews_gmaps['date'] = pd.to_datetime(reviews_gmaps['review_timestamp'], unit='s')
reviews_gmaps['timestamp'] = reviews_gmaps['date'].dt.strftime('%Y-%m-%d %H:%M:%S').astype(str)
reviews_gmaps['hour'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[1].split(':')[0].strip(), axis=1)
reviews_gmaps['year'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[0].split('-')[0].strip(), axis=1)
reviews_gmaps['month'] = reviews_gmaps.apply(lambda row: row.timestamp.split(' ')[0].split('-')[1].strip(), axis=1)
list_centros = list(reviews_gmaps.name.unique())

## Grafico

In [ ]:
#reviews_gmaps = pd.read_csv('./sentimientos/sentiments_arkadia.csv')


In [ ]:
list_centros

In [ ]:
# Suponiendo que list_centros es una lista de nombres de centros y
# reviews_gmaps es un DataFrame que contiene las columnas 'name' y 'hour'

for centro in list_centros:
    # Filtrar el DataFrame para el centro actual en la iteración
    reviews_centro = reviews_gmaps[reviews_gmaps['name'] == centro]
    # Contar los valores únicos en la columna 'hour'
    conteo_valores = reviews_centro['sentiment'].value_counts().sort_index()

    # Crear el gráfico de barras usando Seaborn
    plt.figure(figsize=(10, 6))
    sns.barplot(x=conteo_valores.index, y=conteo_valores.values, color='skyblue')

    # Personalizar el gráfico (etiquetas, título, etc.)
    plt.xlabel('Estrellas')
    plt.ylabel('Frecuencia')
    plt.title(f'Frecuencia de reviews por estrellas para {centro}')

    # Mostrar el gráfico
    plt.savefig(f'./sentimientos/graficos/arkadia/sentimientos/{centro}.jpg', dpi=300)
    #plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que df es tu DataFrame y "datos" es la columna que deseas representar
# Puedes contar los valores únicos en la columna usando value_counts()

conteo_valores = reviews_gmaps['hour'].value_counts().sort_index()

# Crear el gráfico de barras usando Seaborn
plt.figure(figsize=(10, 6))

sns.barplot(x=conteo_valores.index, y=conteo_valores.values, color='orange')

# Personalizar el gráfico (etiquetas, título, etc.)
plt.xlabel('hora')
plt.ylabel('Frecuencia')
plt.title('freq')

# Mostrar el gráfico
plt.show()